# 2. Additive Manufacturing Path Planning Made Effortless

--> ***(before running this code, please consult Quick Start to make sure everything is set up)***

--> You will additionally need `pycalphad` and `pathfinding` libraries to run part of this exercise. If you are running this in Codespaces, it has been pre-installed for you.

**In this tutorial, we will demonstrate how effortless it is to dramatically speed up the exploration of feasible compositional spaces in high dimensional spaces through employing `nimplex`'s graph representations that abstract the underlying problem and dimensionality.**

**We will also design several neat, mathematically optimal (given some criteria) paths in a 7-component chemical space connecting two alloys of interest by mixing 4 fixed-composition alloy powders to create a tetrahedral attainable/design space. The beauty of this approach is that at no point (except for plotting in 3D for "human consumption") will we explicitly consider the dimensionality or the distance as the connectivity between the points in the space has been abstracted into graph adjacency. If you wish to add another alloy to the design process, you add it to the list, and you are done :)**

In [76]:
# Import nimplex and some of its plotting utilities
import nimplex
from utils import plotting

In [77]:
# Python wrapper for Plotly library
import plotly.express as px
import pandas as pd
from pprint import pprint
from copy import deepcopy

## Recall Last Example
Let's get back to our example (QuickStart) of the pair of **attainable space** simplex grid and corresponding **elemental space** positions defined for `7`-component elemental space of `Ti`, `Zr`, `Hf`, `W`, `Nb`, `Ta`, and `Mo` formed by `4` alloys:
- Ti50 Zr50 
- Hf95 Ti5
- Mo33 Nb33 Ta33
- Mo10 Nb10 W80

which we can represent as points in the elemental space:

In [78]:
elementalSpaceComponents = ['Ni', 'Cr', 'Fe']
attainableSpaceComponents = ['SS304L', 'Ni', 'Cr']
attainableSpaceComponentPositions = [[0.096114519430,0.1993865031, 0.7044989775], [100, 0, 0], [0, 100, 0]]

And create tetrahedral grids with their compositions quantized at `12` divisions per dimension.

In [79]:
gridAtt, gridEl = nimplex.embeddedpair_simplex_grid_fractional_py(attainableSpaceComponentPositions, 12)

We used the **elemental**, or chemical, space to run the Root Mean Square Atomic Displacement (RMSAD) model by Tandoc (10.1038/s41524-023-00993-x) which acts as a lower-cost proxy for yield stress and hardness estimations in the absence of direct data:

import pqam_rmsadtandoc2023
rmsadList = []
for point in gridEl:
    formula = ' '.join([f'{c}{p}' for c, p in zip(elementalSpaceComponents, point)])
    rmsadList.append(pqam_rmsadtandoc2023.predict(formula))

And the **attainable** space grid to plot the results after projecting them to the Euclidean space:

# Hover approximate formula for each point
formulas = []
for i, comp in enumerate(gridEl):
    formulas.append(f"({i:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(elementalSpaceComponents, comp)]))

# Generate the projected grid
gridAtt_projected_df = pd.DataFrame(plotting.simplex2cartesian_py(gridAtt), columns=['x','y'])

# Attach pure component (alloy) labels to corners
pureComponentIndices = nimplex.pure_component_indexes_py(3, 12)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(attainableSpaceComponents, pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"

# Add text labels at the corners of the simplex
px.scatter(gridAtt_projected_df, x='x', y='y', color=rmsadList, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=700, 
              labels={'color':'RMSAD', 'x':'', 'y':'', 'z':''})

In [80]:
# Hover approximate formula for each point
formulas = []
for i, comp in enumerate(gridEl):
    formulas.append(f"({i:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(elementalSpaceComponents, comp)]))

# Generate the projected grid
gridAtt_projected_df = pd.DataFrame(plotting.simplex2cartesian_py(gridAtt), columns=['x','y'])

# Attach pure component (alloy) labels to corners
pureComponentIndices = nimplex.pure_component_indexes_py(3, 12)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(attainableSpaceComponents, pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"


## Thermodynamic Equilibria

**In this section we will be using `pycalphad` which is an amazing free open-source library for building thermodynamic models and performing calculations on them, including studying of phase equilibria, or what atomic arrangements are stable (coexist in equilibrium) at a given temperature and composition.** You can read more about `pycalphad` at [pycalphad.org](https://pycalphad.org/) and find an great tutorial at [this 2023 workshop materials repository](https://github.com/materialsgenomefoundation/2023-workshop-material/tree/main/pycalphad).

Let's start by loading a database of thermodynamic properties which defines, among other things, the Gibbs energy of each phase as a function of temperature and composition. You can find many of such databases at [TDBDB](https://avdwgroup.engin.brown.edu) maintained by Axel van de Walle's group at Brown University. 

In this example, we will be using a TDB `CrHfMoNbTaTiVWZr_9element_Feb2023` placed for you in the `examples` directory, which is a 9-element database for the elements `Cr`, `Hf`, `Mo`, `Nb`, `Ta`, `Ti`, `V`, `W`, and `Zr`, being developed by Shuang Lin in our group (Phases Research Lab at PSU). This wersion is an older *work in progress* one that has not been published, so while it is perfect for tutorial like this one, please refrain from using it for any serious work.

In [81]:
from pycalphad import Database
dbf = Database("ammap/databases/Cr-Fe-Ni_miettinen1999.tdb")
phases = list(set(dbf.phases.keys()))
print(elementalSpaceComponents)
print(f'Loaded TDB file with phases considered: {phases}')

['Ni', 'Cr', 'Fe']
Loaded TDB file with phases considered: ['HCP_A3', 'LIQUID', 'FCC_A1', 'BCC_A2', 'SIGMA']


As you can see, we will be looking at several different phases here. To keep things as simple as possible, we can split them in three groups:
- **liquid** phases: `LIQUID` (which we obviously want to avoid in a solid part design)
- **solid solution** phases: `FCC_A1`, `BCC_A2`, and `HCP_A3`
- **intermetallic** phases: `LAVES_C14`, `LAVES_C15`, and `LAVES_C36`

Designing an alloy is a complex procedure but for the sake of this tutorial, we will be focusing on the most common issue which is the formation of intermetallic phases which cause embrittlement and reduce the ductility of the material. **Thus, we will apply a simple constraint that the alloy should only contain the solid solution phases FCC, BCC, and HCP.**

**Now, knowing what we are looking for (phases at equilibrium), we can start by writing a short Python script around `pycalphad` to calculate the equilibrium phases for a given chemical elements composition `elP`. It is already placed in the `examples` directory as `myPycalphadCallable.py` which defines a function `equilibrium_callable` that takes a composition and returns the equilibrium phases.**

**We will arbitrarily pick `1000K`** as the temperature for the sake of this tutorial, but you can change it to any other value you like. Or even make it a list and add phases present at each temperature to a set to apply our constraint over a range of temperatures.


Please note that much more information is generated in the process (e.g., chemical composition of each phase and its fraction) but we are only interested in the phase presence. If you wish to do so, modifying the script to, e.g., allow for up to 5% of intermetallic phases, is a trivial task. Advanced users may also want to have a look at the `scheil_callable` we do not use in this tutorial for the sake of runtime, but which can be used to simulate solidification of the alloy from a liquid state in an additive manufacturing process.

In [82]:
from ammap.callables.EqScheil import equilibrium_callable

Let's test it on some composition in our space starting with the first point!

In [83]:
print(formulas[0])
equilibrium_callable(gridEl[0])

(  0) Cr100.0 


{'Phases': ['BCC_A2'], 'PhaseFraction': [1.0000000000011475]}

You should see `['BCC_A2']` in a second or so if you've run it at the default `1000K`. Quick and neat, right? Now, let's pick some compositionally complex alloy that does not lay around the corner of the attainable space tetrahedron and presents an actual challenge.

In [84]:
print(formulas[22])
equilibrium_callable(gridEl[22])

( 22) Ni75.8 Cr18.3 Fe5.9 


{'Phases': ['FCC_A1'], 'PhaseFraction': [1.0000000000000002]}

Now, you should have seen an example of infeasible point composed of `['HCP_A3', 'LAVES_C15', 'BCC_A2']`. Let's deploy this in parallel over all the points in the elemental space `gridEl` and see how it looks like! We will use the `process_map` function from the `tqdm` library to show a neat progress bar while the calculations are running in parallel. On the 4-core Codespaces VM you can expect it to take around 2-3 minutes.

In [85]:
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map

In [86]:
gridPhases = process_map(equilibrium_callable, gridEl)

  0%|          | 0/91 [00:00<?, ?it/s]

Let's see how some of the data looks like.

In [87]:
len(gridPhases)

91

Now, let's turn that list of phases into a list of feasibility based on the constraint we defined earlier. Note that in some cases, the `pycalphad` library may return an empty list of phases, which we will treat as infeasible.

infeasiblePhases = set([
    'AL3M', 'AL5FE2', 'LIQUID', 'AL11CR4', 'D82_217', 'AL3NI1', 'NI3TI', 'AL5FE4', 'TAOA', 'LAVES_C15', 'GAMMAH', 'AL5CR', 'AL2FE', 'AL4CR', 'NIV3_A15', 'GAMMA_H', 
    'ALTI3_D019', 'ALCU_DELTA', 'AL45CR7', 'TS01TI', 'NI8V', 'AL45CR7_12', 'LAVES_C14', 'LAVES_A15', 'AL3NI5', 'AL5TI3', 'ALCU_EPSILON', 'NITI2', 'AL8V5', 'ALCR2', 'AL23V4', 'AL13FE4', 
    'AL4NI3', 'ALCU_ZETA', 'ALCU_THETA', 'AL3NI2', 'AL23V4_194', 'CRNI2', 'NI3V_D022', 'LAVES_C36', 'TS01T3', 'AL21V2_227', 'AL5TI2', 'ALCU_ETA', 'GAMMAL', 
    'GAMMA_D83', 'SIGMA_D8B', 'AL21V2', 'AL2TI', 'AL3TI_L', 'ALTI', 'AL45V7'])
print(f"Feasible phases set: {set(phases).difference(infeasiblePhases)}")


In [88]:
infeasiblePhases = set(['LIQUID', 'SIGMA'])
print(f"Feasible phases set: {set(phases).difference(infeasiblePhases)}")

Feasible phases set: {'FCC_A1', 'HCP_A3', 'BCC_A2'}


In [89]:
gridFeasible = [len(set(p) & set(['SIGMA', 'LIQUID']))==0 and p!=[] for p in gridPhases]
gridFeasible[20:30]

[True, True, True, True, True, True, True, True, True, True]

In [90]:
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasible, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=600, color_discrete_sequence=['green', 'red'], 
              labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

In [91]:
for phase in gridPhases:
    phase['infeasiblePhases'] = sum(phase['PhaseFraction'][i] for i, p in enumerate(phase['Phases']) if p in infeasiblePhases)
phases1 = [gridPhases[i]['Phases'] for i in range(len(gridPhases))]
sum1= [gridPhases[i]['infeasiblePhases'] for i in range(len(gridPhases))]
gridFeasible = [len(set(p) & infeasiblePhases)==0 and p!=[] for p in phases1]

In [92]:
gridFeasible = [len(set(p) & infeasiblePhases)==0 and p!=[] for p in phases1]


In [93]:
len(gridFeasible)

91

Finally, let's plot the result in 3D using the `plotly` library and our spacial-transformed attainable space grid we obtained with `plotting.simplex2cartesian_py(gridAtt)` earlier.

**Once you run the cell below, you should be seeing an interactive 3D plot with 455 split roughly 50/50 between feasible and infeasible points. You can rotate the plot, zoom in and out, and hover over the points to see their composition and feasibility. You can also click on the legend to hide/show the points based on their feasibility.**

In [94]:
color_map = {'path':'red', True:'#FECB52', False:'blue'}

In [95]:
gridFeasible[90]=True

In [96]:
gridFeasible
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasible, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=600, color_discrete_map=color_map, 
              labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.update_traces( marker_size=12)
fig.show()

In [97]:
from pathfinding.core.graph import Graph
from pathfinding.finder.dijkstra import DijkstraFinder

In [98]:
finder = DijkstraFinder()

## Multi-TDB

- elementalSpaceComponents = ["Ti", "Zr", "Hf", "W", "Nb", "Ta", "Mo"]
### System 1
- attainableSpaceComponents = ["Ti50 Zr50", "Mo33 Nb33 Ta33", "Mo80 Nb10 W10"]
- attainableSpaceComponentPositions = [[50, 50, 0, 0, 0, 0, 0], [0, 0, 0, 33, 33, 33, 0], [0, 0, 0, 10, 10, 0, 80]]
### System 2
- attainableSpaceComponents = ["Ti50 Zr50", "Mo33 Nb33 Ta33", "Hf95 Zr5"]
- attainableSpaceComponentPositions = [[50, 50, 0, 0, 0, 0, 0], [0, 0, 0, 33, 33, 33, 0], [0, 5, 95, 0, 0, 0, 0]]

In [99]:
def system_graphs(systemIdx, attainableSpaceComponentPositions, ndiv=12):
    gridAtt, gridEl, graphN = nimplex.embeddedpair_simplex_graph_fractional_py(attainableSpaceComponentPositions, ndiv)
    gridPhases = process_map(equilibrium_callable, gridEl)
    gridFeasible = [len(set(p) & infeasiblePhases)==0 and p!=[] for p in gridPhases]
    edges = []
    offset = len(graphN)*systemIdx
    for i, nList in enumerate(graphN):
        if gridFeasible[i]:
            for n in nList:
                if gridFeasible[n]:
                    edges.append([i+offset, n+offset, 1])
    return gridAtt, gridEl, graphN, edges, gridFeasible, gridPhases

In [100]:
gridAtt1, gridEl1, graphN1, edges1, gridFeasible1, gridPhases1 = system_graphs(0, [[100, 0, 0], [0, 100, 0],[0.096114519430,0.1993865031, 0.7044989775]])

  0%|          | 0/91 [00:00<?, ?it/s]

In [101]:
for phase in gridPhases1:
    phase['infeasiblePhases'] = sum(phase['PhaseFraction'][i] for i, p in enumerate(phase['Phases']) if p in infeasiblePhases)
phases1 = [gridPhases1[i]['Phases'] for i in range(len(gridPhases1))]
sum1= [gridPhases1[i]['infeasiblePhases'] for i in range(len(gridPhases1))]
gridFeasible11 = [len(set(p) & infeasiblePhases)==0 and p!=[] for p in phases1]

In [102]:
gridFeasible11[0]=True

In [127]:
pureComponentIndices = nimplex.pure_component_indexes_py(3, 12)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(["Ni", "Cr", "SS304L"], pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasible11, text=labels, 
              template='plotly_white', width=800, height=600, color_discrete_map=color_map, 
              labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

In [175]:
from ammap.callables.EqScheil2 import equilibrium_callable2
from ammap.callables.EqScheil1 import equilibrium_callable1
from ammap.callables.EqScheil1 import elementalSpaceComponents as comp1

In [129]:
def system_graphs1(systemIdx, attainableSpaceComponentPositions, ndiv=12):
    gridAtt, gridEl, graphN = nimplex.embeddedpair_simplex_graph_fractional_py(attainableSpaceComponentPositions, ndiv)
    gridPhases = process_map(equilibrium_callable1, gridEl)
    gridFeasible = [len(set(p) & infeasiblePhases)==0 and p!=[] for p in gridPhases]
    edges = []
    offset = len(graphN)*systemIdx
    for i, nList in enumerate(graphN):
        if gridFeasible[i]:
            for n in nList:
                if gridFeasible[n]:
                    edges.append([i+offset, n+offset, 1])
    return gridAtt, gridEl, graphN, edges, gridFeasible, gridPhases, spaceComps=elementalSpaceComponents

def system_graphs2(systemIdx, attainableSpaceComponentPositions, ndiv=12):
    gridAtt, gridEl, graphN = nimplex.embeddedpair_simplex_graph_fractional_py(attainableSpaceComponentPositions, ndiv)
    gridPhases = process_map(equilibrium_callable2, gridEl)
    gridFeasible = [len(set(p) & infeasiblePhases)==0 and p!=[] for p in gridPhases]
    edges = []
    offset = len(graphN)*systemIdx
    for i, nList in enumerate(graphN):
        if gridFeasible[i]:
            for n in nList:
                if gridFeasible[n]:
                    edges.append([i+offset, n+offset, 1])
    return gridAtt, gridEl, graphN, edges, gridFeasible, gridPhases

In [130]:
gridAtt2, gridEl2, graphN2, edges2, gridFeasible2, gridPhases2 = system_graphs1(1, [[0,100,0], [100,0,0], [0,0,100]])

  0%|          | 0/91 [00:00<?, ?it/s]

In [131]:
for phase in gridPhases2:
    phase['infeasiblePhases'] = sum(phase['PhaseFraction'][i] for i, p in enumerate(phase['Phases']) if p in infeasiblePhases)
phases1 = [gridPhases2[i]['Phases'] for i in range(len(gridPhases2))]
sum1= [gridPhases2[i]['infeasiblePhases'] for i in range(len(gridPhases2))]
gridFeasible22 = [len(set(p) & infeasiblePhases)==0 and p!=[] for p in phases1]

In [132]:
pureComponentIndices = nimplex.pure_component_indexes_py(3, 12)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(["Ni", "Cr", "V"], pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasible22, text=labels, 
              template='plotly_white', width=800, height=600, color_discrete_map=color_map, 
              labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

In [133]:
def stitch_edge(excludedComponent1Idx, excludedComponent2Idx, systemIdx1, systemIdx2, ndiv=12):
    edge1, edge2 = [], []
    stitchEdges = []
    gridAtt = nimplex.simplex_grid_py(3, ndiv)
    offset1 = len(gridAtt)*systemIdx1
    offset2 = len(gridAtt)*systemIdx2
    for i, p in enumerate(gridAtt):
        if p[excludedComponent1Idx]==0:
            edge1.append(i+offset1)
        if p[excludedComponent2Idx]==0:
            edge2.append(i+offset2)
    assert len(edge1) == len(edge2)
    assert len(edge1)>0
    print(edge1)
    print(edge2)
    for i, j in zip(edge1, edge2):
        stitchEdges.append([i,j,1])
        stitchEdges.append([j,i,1])
    print(stitchEdges)
    return stitchEdges

In [166]:
stitch12 = stitch_edge(2,2,1,0)

[103, 115, 126, 136, 145, 153, 160, 166, 171, 175, 178, 180, 181]
[12, 24, 35, 45, 54, 62, 69, 75, 80, 84, 87, 89, 90]
[[103, 12, 1], [12, 103, 1], [115, 24, 1], [24, 115, 1], [126, 35, 1], [35, 126, 1], [136, 45, 1], [45, 136, 1], [145, 54, 1], [54, 145, 1], [153, 62, 1], [62, 153, 1], [160, 69, 1], [69, 160, 1], [166, 75, 1], [75, 166, 1], [171, 80, 1], [80, 171, 1], [175, 84, 1], [84, 175, 1], [178, 87, 1], [87, 178, 1], [180, 89, 1], [89, 180, 1], [181, 90, 1], [90, 181, 1]]


In [167]:
edgesCombined = edges1 + edges2 + stitch12

In [168]:
pathfindingGraph = Graph(edges=deepcopy(edgesCombined), bi_directional=False)

In [169]:
finder = DijkstraFinder()

In [170]:
path, runs = finder.find_path(
    pathfindingGraph.node(0), 
    pathfindingGraph.node(91),
    pathfindingGraph)

In [171]:
shortestPath = [p.node_id for p in path]

In [176]:
formulas = []
for i, comp in enumerate(gridEl1):
    formulas.append(f"({i:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(elementalSpaceComponents, comp)]))
for i, comp in enumerate(gridEl2):
    formulas.append(f"({i+91:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(comp1, comp)]))

In [177]:
for step, i in enumerate(shortestPath):
    print(f"{step+1:>2}: {formulas[i]}")

 1: (  0) Ni9.6 Cr19.9 Fe70.4 
 2: (  1) Ni8.8 Cr26.6 Fe64.6 
 3: (  2) Ni8.0 Cr33.3 Fe58.7 
 4: (  3) Ni7.2 Cr40.0 Fe52.8 
 5: (  4) Ni6.4 Cr46.6 Fe47.0 
 6: (  5) Ni5.6 Cr53.3 Fe41.1 
 7: (  6) Ni4.8 Cr60.0 Fe35.2 
 8: (  7) Ni4.0 Cr66.6 Fe29.4 
 9: (  8) Ni3.2 Cr73.3 Fe23.5 
10: (  9) Ni2.4 Cr80.0 Fe17.6 
11: ( 10) Ni1.6 Cr86.7 Fe11.7 
12: ( 11) Ni0.8 Cr93.3 Fe5.9 
13: ( 12) Cr100.0 
14: (103) Cr100.0 
15: (102) Cr91.7 V8.3 
16: (101) Cr83.3 V16.7 
17: (100) Cr75.0 V25.0 
18: ( 99) Cr66.7 V33.3 
19: ( 98) Cr58.3 V41.7 
20: ( 97) Cr50.0 V50.0 
21: ( 96) Cr41.7 V58.3 
22: ( 95) Cr33.3 V66.7 
23: ( 94) Cr25.0 V75.0 
24: ( 93) Cr16.7 V83.3 
25: ( 92) Cr8.3 V91.7 
26: ( 91) V100.0 


#ISSUES ABOVE

Why is the pathfinding suddenly ignoring the feasibility results?

In [178]:
gridFeasible3=gridFeasible1
gridFeasible4=gridFeasible2

In [183]:
len(gridFeasible3), len(gridFeasible4)

(91, 91)

In [184]:
gridFeasibleMarked3 = ['path' if i in shortestPath else f for i, f in enumerate(gridFeasible11)]

In [185]:
gridFeasibleMarked4 = ['path' if i in shortestPath else f for i, f in enumerate(gridFeasible22, start=91)]

In [186]:
pureComponentIndices = nimplex.pure_component_indexes_py(3, 12)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(["Ni", "Cr", "SS"], pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasibleMarked4, text=labels, 
              template='plotly_white', width=800, height=600, color_discrete_map=color_map)#, 
              #labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

In [189]:
pureComponentIndices = nimplex.pure_component_indexes_py(3, 12)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(comp1, pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasibleMarked3, text=labels, 
              template='plotly_white', width=800, height=600, color_discrete_map=color_map) 
              #labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

In [125]:
# Initialize the list to store the enumerated path as tuples
enumeratedPath = []

# Iterate through shortestPath and save the step and formula as tuples
for step, i in enumerate(shortestPath):
    enumeratedPath.append((step + 1, formulas[i]))

# Now enumeratedPath contains tuples with the step number and the corresponding formula
print(enumeratedPath)

[(1, '(  0) Ni9.6 Cr19.9 Fe70.4 '), (2, '( 91) Fe100.0 ')]


In [126]:
import csv

# Assuming enumeratedPath is already defined
# Example: enumeratedPath = [(1, 'O2'), (2, 'CH4'), (3, 'C3H8')]

# Define the CSV file name
csv_file = 'enumerated_path.csv'

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header (optional)
    writer.writerow(['Step', 'Formula'])
    
    # Write the data
    writer.writerows(enumeratedPath)

print(f"Data has been exported to {csv_file}")

Data has been exported to enumerated_path.csv
